<div class="alert alert-block alert-success">
Do the basic imports 
</div>

In [1]:
!pip install beautifulsoup4

In [49]:
import pandas as pd
import numpy as np

In [2]:
import requests
from bs4 import BeautifulSoup

<div class="alert alert-block alert-success">
Load the BeautifulSoup object with Wiki data and replace line breaks for easier parsing
</div>

In [ ]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = requests.get(wiki_url).text
soup = BeautifulSoup(page, "html.parser")

# REPLACE <BR> WITH PIPE FOR PARSING
delimiter = '|'                           # unambiguous string
for line_break in soup.findAll('br'):       # loop through line break tags
    line_break.replaceWith(delimiter) 

<div class="alert alert-block alert-success">
Create a function to clean the neighborhood data
</div>

In [143]:
def special_replace(neigh):
    result = neigh.replace('(','')
    result = result.replace(')','')
    result = result.strip()
    return result

<div class="alert alert-block alert-success">
Create a dataframe to hold data. Parse out the data, omitting cases where no borough is listed. Sort the dataframe by borough and postal code.
</div>

In [146]:
data = {"PostalCode":"XXX", "Borough":"XXX", "Neighborhood":"XXX"}
tor_neigh = pd.DataFrame(columns=["PostalCode","Borough","Neighborhood"], index = [0]) 

tbl = soup.find_all('table')[0]

use_anchor_method = False
row_marker = 0

# Big Parsing Loop - Rows, then Cells
for row in tbl.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    # each td has a p with the postal code, a span which can say Not assigned, or a span with <a> elements,
    # the first is the borough, all the rest of the items after split are the neighborhoods
    for column in columns:
        code = column.find_all('p')[0].get_text()[:3]
        span = column.find_all('span')[0]
        a_count = len(span.find_all('a')) 
        if span.get_text() == "Not assigned":
            continue
        else: 
            special_neighs = span.get_text().replace('/','|').split('|')
            borough = special_neighs[0]
            for n in range(1, len(special_neighs)):
                newneigh = special_neighs[n]
                if neigh == np.nan or neigh == None or neigh == "":
                    neigh = special_replace(newneigh)
                else:
                    neigh += ", " + special_replace(newneigh)
                        
            new_row = {'PostalCode':code,'Borough':borough,'Neighborhood':neigh}
            tor_neigh = tor_neigh.append(new_row, ignore_index=True)
        
        # Clear these values so we can't contaminate the next pass
        borough = ""
        neigh = ""
        newneigh = ""
        column_marker += 1

tor_neigh.drop(labels=0,axis=0,inplace=True)
tor_neigh.sort_values(by=["Borough", "PostalCode"], inplace=True)
tor_neigh = tor_neigh.reset_index(drop=True)
tor_neigh[1:51]

,PostalCode,Borough,Neighborhood
1,M4P,Central Toronto,Davisville North
2,M4R,Central Toronto,North Toronto West
3,M4S,Central Toronto,Davisville
4,M4T,Central Toronto,"Moore Park, Summerhill East"
5,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
6,M5N,Central Toronto,Roselawn
7,M5P,Central Toronto,Forest Hill North & West
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
9,M4W,Downtown Toronto,Rosedale
10,M4X,Downtown Toronto,"St. James Town, Cabbagetown"


In [147]:
tor_neigh.shape

(103, 3)

<div class="alert alert-block alert-success">
<b>I DID NOT FIND THIS TO BE TRUE</b><br>
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  
in the above table.
</div>

In [148]:
df_pc_count = tor_neigh.groupby('PostalCode').size().reset_index()
df_pc_count.columns = ["PostalCode","CodeCount"]
df_pc_count[df_pc_count["CodeCount"] > 1]

,PostalCode,CodeCount


In [149]:
tor_neigh[tor_neigh["PostalCode"]=="M5A"]

,PostalCode,Borough,Neighborhood
12,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [100]:
tor_neigh.tail()

,PostalCode,Borough,Neighborhood
99,M8X,Etobicoke,"The Kingsway, Old Mill"
100,M4Y,Downtown Toronto,Church and Wellesley
101,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
102,M8Y,Etobicoke,"Old Mill, Sunnylea, Humber Bay, Mimico, The Qu..."
103,M8Z,Etobicoke,"Mimico, The Queensway"
